In [1]:
# Importar los datos en un dataframe
import pandas as pd
import numpy as np

data = pd.read_csv('loan.csv',low_memory=False)
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Obtener las estadisticas descriptivas
pd.set_option('display.max_columns', 500)
data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,8.873790e+05,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,8.873750e+05,887379.000000,887350.000000,887350.000000,433067.000000,137053.000000,887350.000000,887350.000000,8.873790e+05,886877.000000,887350.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,887234.000000,221703.000000,887379.0,511.000000,509.000000,887350.000000,8.171030e+05,8.171030e+05,21372.000000,21372.000000,21372.000000,21372.000000,20810.000000,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171030e+05,21372.000000,21372.000000,21372.000000
mean,3.246513e+07,3.500182e+07,14755.264605,14741.877625,14702.464383,13.246740,436.717127,7.502759e+04,18.157039,0.314442,0.694623,34.063798,70.117903,11.548469,0.195307,1.692079e+04,55.067693,25.268026,8403.124852,8399.713242,7558.826684,7521.160306,5757.706424,1754.804327,0.396692,45.919243,4.880757,2164.145585,0.014380,44.104838,1.0,109981.011585,18.310118,0.004991,2.257026e+02,1.394582e+05,1.109021,2.928832,0.761651,1.674574,20.912686,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206862e+04,0.943945,1.537058,1.977307
std,2.282734e+07,2.411335e+07,8435.455601,8429.897657,8442.106732,4.381867,244.186593,6.469830e+04,17.190626,0.862244,0.998448,21.884940,28.127914,5.317313,0.582091,2.242679e+04,23.834344,11.840561,8493.450269,8489.997617,7871.243336,7846.108604,6625.441046,2095.356099,4.087825,409.693874,63.125281,4794.783233,0.134191,22.179841,0.0,52730.379847,7.169233,0.077625,1.031137e+04,1.537500e+05,1.242675,3.089987,0.996035,1.688725,27.209081,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749826e+04,1.446872,2.723339,2.874067
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.320000,15.670000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,17950.000000,3.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,9.206643e+06,1.087713e+07,8000.000000,8000.000000,8000.000000,9.990000,260.705000,4.500000e+04,11.910000,0.000000,0.000000,15.000000,51.000000,8.000000,0.000000,6.443000e+03,37.700000,17.000000,0.000000,0.000000,1914.590000,1899.700000,1200.570000,441.460000,0.000000,0.000000,0.000000,280.170000,0.000000,27.000000,1.0,76032.500000,13.200000,0.000000,0.000000e+00,2.985300e+04,0.000000,1.000000,0.000000,0.000000,6.000000,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,3.443327e+07,3.709528e+07,13000.000000,13000.000000,13000.000000,12.990000,382.550000,6.500000e+04,17.650000,0.000000,0.000000,31.000000,70.000000,11.000000,0.000000,1.187500e+04,56.000000,24.000000,6458.290000,6455.810000,4894.999117,4862.010000,3215.320000,1073.280000,0.000000,0.000000,0.000000,462.780000,0.000000,44.000000,1.0,101771.000000,17.610000,0.000000,0.000000e+00,8.055900e+04,1.000000,2.000000,0.000000,1.000000,12.000000,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.0000

In [3]:
# Se eliminan las columnas con mas de 75% de sus valores perdidos (nulos)
cols_elim = [x for x in data.columns if data[x].isnull().sum()/data.count().max()>=0.25]
data_clear_1 = data.drop(columns=cols_elim,axis=1)
print(f'Cantidad de columnas originales: {data.shape[1]}')
print(f'Cantidad de columnas a eliminar: {len(cols_elim)}')
print(f'Cantidad de columnas finales: {data_clear_1.shape[1]}')

Cantidad de columnas originales: 74
Cantidad de columnas a eliminar: 22
Cantidad de columnas finales: 52


In [4]:
# Explorar los posibles valores del target
data_clear_1.loan_status.unique().tolist()

['Fully Paid',
 'Charged Off',
 'Current',
 'Default',
 'Late (31-120 days)',
 'In Grace Period',
 'Late (16-30 days)',
 'Does not meet the credit policy. Status:Fully Paid',
 'Does not meet the credit policy. Status:Charged Off',
 'Issued']

In [5]:
# Se excluyen una serie de estados: Current e Issued porque el prestamo en esos casos aun esta en evaluacion (no es ni positivo ni negativo)
# Se eliminan los estados "Does not meet the credit policy (...)" porque hay incertidumbre de su validez.
estados_eliminar = {'Current','Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off', 'Issued'}
for estado in estados_eliminar:
    data_clear_1.drop(data_clear_1[data_clear_1.loan_status==estado].index,inplace=True)
    print(f'Eliminado {estado}. Quedan: {len(data_clear_1)}')

Eliminado Issued. Quedan: 878919
Eliminado Does not meet the credit policy. Status:Fully Paid. Quedan: 876931
Eliminado Does not meet the credit policy. Status:Charged Off. Quedan: 876170
Eliminado Current. Quedan: 274391


In [6]:
# Buscar las columnas que tienen datos potencialmente categoricos y convertir sus valores de acuerdo al mismo
cat_cols = data_clear_1.select_dtypes(include='object').columns.tolist()
cat_cols.remove('loan_status')
cat_dict = dict()
for col in cat_cols:
    data_clear_1[col] = data_clear_1[col].astype('category')
    cat_dict[col] = {i+1:c for i,c in enumerate(data_clear_1[col].cat.categories)}
    data_clear_1[col] = data_clear_1[col].cat.codes+1

In [17]:
# Se mira la cantidad de categorias que tiene cada columna. Asimismo, se identifican aquellas que tienen una cantidad
# muy alta de categorias distintas (>10% de la cantidad de registros de la columna)
debug = False
cols_elim_2 = []
for keys,values in cat_dict.items():
    if debug:
        print(f'{keys} => {len(values)} de {data_clear_1.count().max()} ({round(100*len(values)/data_clear_1.count().max(),2)}%)')
    if len(values)/data_clear_1.count().max() >= 0.1:
        cols_elim_2.append(keys)
# Los campos de id y member_id no son utiles como caracteristicas, por lo que tambien se agregan a la lista por eliminar
# Y se eliminan los campos
cols_elim_2.append('id')
cols_elim_2.append('member_id')
data_clear_2 = data_clear_1.drop(columns=cols_elim_2, axis=1)
print(f'Cantidad antigua de columnas: {len(data_clear_1.columns.tolist())}')
print(f'Cantidad nueva de columnas: {len(data_clear_2.columns.tolist())}')

Cantidad antigua de columnas: 52
Cantidad nueva de columnas: 47


In [18]:
# Se eliminan las filas con mas de un 25% de vacios
limite=int(data_clear_2.count().count()*0.75+0.9)
print(f'Se eliminaran filas con menos de {limite} columnas no nulas')
print(f'Cantidad de filas original: {len(data_clear_2)}')
data_clear_2 = data_clear_2.dropna(thresh=limite,axis=0)
print(f'Cantidad de filas actual: {len(data_clear_2)}')

Se eliminaran filas con menos de 36 columnas no nulas
Cantidad de filas original: 274391
Cantidad de filas actual: 274391


In [19]:
# Se rellenan los vacios con la mediana
data_clear_3 = data_clear_2.fillna(value=data_clear_2.median())
data_clear_3 = data_clear_3.reset_index(drop=True)
data_clear_3.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,5000.0,5000.0,4975.0,1,10.65,162.87,2,7,2,6,24000.0,3,22,Fully Paid,1,2,772,4,27.65,0.0,245,1.0,3.0,0.0,13648.0,83.7,9.0,1,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,40,171.62,42,0.0,1.0,1,0.0,0.0,78674.0,22010.0
1,2500.0,2500.0,2500.0,2,15.27,59.83,3,14,11,6,30000.0,2,22,Charged Off,1,1,289,11,1.00,0.0,39,5.0,3.0,0.0,1687.0,9.4,4.0,1,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,6,119.66,100,0.0,1.0,1,0.0,0.0,78674.0,22010.0
2,2400.0,2400.0,2400.0,1,15.96,84.33,3,15,2,6,12252.0,1,22,Fully Paid,1,12,546,15,8.72,0.0,529,2.0,2.0,0.0,2956.0,98.5,10.0,1,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,56,649.91,42,0.0,1.0,1,0.0,0.0,78674.0,22010.0
3,10000.0,10000.0,10000.0,1,13.49,339.31,3,11,2,6,49200.0,2,22,Fully Paid,1,10,812,5,20.00,0.0,197,1.0,10.0,0.0,5598.0,21.0,37.0,1,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,40,357.48,41,0.0,1.0,1,0.0,0.0,78674.0,22010.0
4,5000.0,5000.0,5000.0,1,7.90,156.46,1,4,4,6,36000.0,2,22,Fully Paid,1,14,765,4,11.20,0.0,532,3.0,9.0,0.0,7963.0,28.3,12.0,1,0.0,0.0,5631.377753,5631.38,5000.00,631.38,0.00,0.00,0.00,40,161.03,102,0.0,1.0,1,0.0,0.0,78674.0,22010.0


In [20]:
# Se genera la variable de destino (target). En este caso, el estado de prestamo pagado será la clase positiva (+1)
# y cualquier otro estado será negativo (-1)
data_clear_3['target'] = 2*(data_clear_3['loan_status'] == 'Fully Paid')-1
y = data_clear_3['target']

In [21]:
# Se separa el vector de caracteristicas X
X = data_clear_3[data_clear_3.drop(columns=['loan_status','target'],axis=1).columns.tolist()]
X.head(2)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,5000.0,5000.0,4975.0,1,10.65,162.87,2,7,2,6,24000.0,3,22,1,2,772,4,27.65,0.0,245,1.0,3.0,0.0,13648.0,83.7,9.0,1,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,40,171.62,42,0.0,1.0,1,0.0,0.0,78674.0,22010.0
1,2500.0,2500.0,2500.0,2,15.27,59.83,3,14,11,6,30000.0,2,22,1,1,289,11,1.00,0.0,39,5.0,3.0,0.0,1687.0,9.4,4.0,1,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,6,119.66,100,0.0,1.0,1,0.0,0.0,78674.0,22010.0


In [22]:
# Separar el conjunto de entrenamiento/validacion del conjunto de prueba
from sklearn.model_selection import train_test_split

X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

In [23]:
# Normalizar el conjunto de entrenamiento/valdiacion
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_trainval)
X_trainval_scaled = scaler.transform(X_trainval)

In [25]:
# Uso de pipes para entrenar modelo y seleccionar caracteristicas con GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

modelo_logistico = LogisticRegression()
selector_logistico = RFE(modelo_logistico)
pipe = Pipeline(steps=[('reducir_dim', selector_logistico), ('logistic', modelo_logistico)])

n_caracteristicas = list(range(1,len(X_trainval.columns)))
c_valores = [1, 10, 100, 1000]

param_grid = [
    {
        'reducir_dim__n_features_to_select': n_caracteristicas,
        'logistic__C': c_valores
    }
]

grid = GridSearchCV(pipe, cv=3, n_jobs=1, param_grid=param_grid)
grid.fit(X_trainval,y_trainval)

KeyboardInterrupt: 